In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas

import json
import mysql.connector
from datetime import datetime
from services.db_ops import helpers as db_hlp, query_factory as qf, db as db_ops
from services.ai_chat import agents
from services.pql.translate import  PQLTranslator


env_path = Path('../services/docker/.env')
load_dotenv(dotenv_path=env_path)

True

In [15]:
table_name = "Orders_oct_16_2024_20_00"
eval_name = "eval_oct_16_2024_20_00"
reset_data = False
eval_config = {}
with open('services/pql/eval/eval_config.json', 'r') as file:
    eval_config = json.load(file)
    eval_config = eval_config.get(eval_name)

In [9]:
print(eval_config['question_generation_prompt'])

This is data for testing an app to generate SQL to answer user queries. Using these data and questions you generate, large language models will generate SQL and be evaluated on the result. The target audience for the app is data analysts. The scope is business data, so the target audience will be doing business analysis. Currently the app answers only single-value questions (questions that can be answered by single value like a KPI instead of multiple values like a chart). The app supports 5 data types: string, integer, float, date, and null. To evaluate the app, we need to generate questions that can be answered by SQL queries on this data. The questions should be in different complexities - from level_0 to level_5. Please generate three valid questions of each complexity level and three invalid questions of each complexity level. Please provide the the questions in JSON. For each, use {question: ---, expected_value: ---} format, and leave expected_value empty


In [10]:
mysql_root_user = os.getenv("MYSQL_ROOT_USER")
mysql_root_password = os.getenv("MYSQL_ROOT_PASSWORD")
mysql_database = os.getenv("MYSQL_DATABASE")

openai_key = os.getenv("OPENAI_API_KEY")
openai_model = os.getenv(eval_config["model"])
openai_assistant_id = os.getenv("OPEN_AI_ANALYSIS_CHAT_ASSISTANT_ID")

In [11]:
mydb = mysql.connector.connect(
    host="localhost",
    user=mysql_root_user,
    password=mysql_root_password,
    database=mysql_database
)

mycursor = mydb.cursor()

In [12]:
df = pandas.read_csv(f"services/pql/eval/eval_data/{table_name}.csv")
df = df.where(pandas.notnull(df), None) # Replace NaN with None for MySQL
data = df.to_dict(orient='records')

table_meta = None
with open('services/pql/eval/eval_data/data_config.json', 'r') as file:
    _all_columns = json.load(file)
    table_meta = _all_columns.get(table_name)
    columns = table_meta.get("columns")

assert columns is not None, "Columns not loaded"
assert isinstance(columns, dict), "Columns is not a dict"

columns_with_dtype = []
for k, v in columns.items():
    columns_with_dtype.append((k, db_hlp.DATA_TYPE_MAP[v['dtype']]))

In [13]:
# Get the table and column information
_table_information = f"""Table information:\n
Table name: {table_name if eval_config['table_name_provided'] else ''}\n
Table description: {table_meta['table_description'] if eval_config['table_descrition_provided'] else ''}\n
"""

_column_information = "Column information:\n"
for k, v in columns.items():
    _column_information += f"""Column name: {k if eval_config['column_names_provided'] else ''}\n
    Column description: {v['description'] if eval_config['column_descriptions_provided'] else ''}\n
    Column data type: {v['dtype'] if eval_config['column_dtypes_provided'] else ''}\n
"""

In [14]:
if reset_data:
    # create table
    query = f'''
        DROP TABLE IF EXISTS `{table_name}`;
    '''
    mycursor.execute(query)
    mydb.commit()

    query = qf.generate_create_table_sql(table_name, columns_with_dtype)[0]
    mycursor.execute(query)
    mydb.commit()

    # insert data
    column_formats = {}
    for k, v in columns.items():
        column_formats[k] = v['format']
    _query, _inputs = qf.generate_insert_data_sql(table_name, data, column_formats)
    mycursor.execute(_query, _inputs)
    mydb.commit()

In [8]:
eval_result_data_name = datetime.now().isoformat().split(".")[0]
eval_result_file_name = f"services/pql/eval/eval_results/{eval_result_data_name}.json"
runs = {
    "pql_version": "v0.1.0",
    "eval_time": datetime.now().isoformat(),
    "eval_config": eval_config,
    "assistant": openai_assistant_id,
    "runs": {
        "simple_questions": [],
        "medium_questions": [],
        "complex_questions": []
    }
}

In [ ]:
valid_questions = table_meta.get("user_questions").get("valid_questions")

In [9]:
for level in list(valid_questions.keys()):
    for q_info in valid_questions[level]:
        question_category = "valid_questions"
        question = q_info.get("question")
        expected_value = q_info.get("expected_value")
        
        chat_id = f"{table_name}_{level}_{q_info.get('index')}"
        openai_agent = agents.OpenAIAnalysisAgent(user_message=question, chat_id=chat_id, thread_id=chat_id)
        openai_agent.start_new_thread()
        result = openai_agent.send_message(table_informaiton=_table_information, column_informaion=_column_information)

        if result.get("success", None) == True:
            # get sql
            _pql = result.get("message")

            assert _pql is not None, "PQL not generated"
            assert isinstance(_pql, dict), "PQL is not a dict"

            _sql_translator = PQLTranslator(pql=_pql)
            _sql_translator.translate()
            result["sql"] = _sql_translator.translated_pql

            # Execute the SQL
            query = result["sql"]
            _sql_executor = db_ops.TranslatedPQLExecution(translated_sql=query)
            print(_sql_executor.translated_sql)

            try:
                _sql_exec_status = True
                mycursor.execute(_sql_executor.translated_sql)
                _sql_exec_result = db_hlp.dictfetchall(mycursor)
            except Exception as e:
                _sql_exec_status = False
                _sql_exec_result = str(e)

            result["sql_execution_status"] = _sql_exec_status
            result["sql_execution_result"] = _sql_exec_result
            result["sql_execution_value"] = list(_sql_exec_result[0].values())[0] if _sql_exec_status else None
            result["expected_value"] = q.get("expected_value")


        runs["runs"][q_type].append(result)

Full agent response: [Message(id='msg_5KirtQKVthyTBEo5758NtZtW', assistant_id='asst_1Z7wPRRU5nxP7P7BYWyXX25m', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='```json\n{\n    "VERSION": "v0.1.0",\n    "NAME": "Total Price",\n    "DESCRIPTION": "The total price of all orders. Calculated by taking the sum of the price column",\n    "TABLE": "Orders_oct_12_2024_20_00",\n    "BLOCKS": [\n        {\n            "SCALAR_BLOCK": {\n                "COLUMNS": ["price"],\n                "OPERATOR": "COLUMN_SUM",\n                "AS": "total_price"\n            }\n        }\n    ]\n}\n```'), type='text')], created_at=1728767305, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_4Z3m0gPIKcmIcX0sthith7MC', status=None, thread_id='thread_o0NxadqORA2zoDzyswKwNjAq'), Message(id='msg_nuBAWwupEYlR0vU3pxRk3dSK', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(tex

In [10]:
runs

{'pql_version': 'v0.1.0',
 'eval_time': '2024-10-12T22:08:20.602338',
 'eval_config': {'model': 'OPEN_AI_MODEL',
  'table_name_provided': True,
  'table_descrition_provided': False,
  'column_names_provided': True,
  'column_descriptions_provided': False,
  'column_dtypes_provided': True},
 'assistant': 'asst_1Z7wPRRU5nxP7P7BYWyXX25m',
 'runs': {'simple_questions': [{'success': True,
    'message': {'VERSION': 'v0.1.0',
     'NAME': 'Total Price',
     'DESCRIPTION': 'The total price of all orders. Calculated by taking the sum of the price column',
     'TABLE': 'Orders_oct_12_2024_20_00',
     'BLOCKS': [{'SCALAR_BLOCK': {'COLUMNS': ['price'],
        'OPERATOR': 'COLUMN_SUM',
        'AS': 'total_price'}}]},
    'chat_id': 1,
    'thread_id': 'thread_o0NxadqORA2zoDzyswKwNjAq',
    'full_conversation': [{'initial_user_message': 'What is the total price?'},
     {'enhanced_user_message': 'Please provide me with a PQL in JSON format that can answer this question, keep it in one JSON and

In [11]:
def datetime_converter(o):
    if isinstance(o, datetime):
        return o.isoformat()
    
with open(eval_result_file_name, 'w') as json_file:
    json.dump(runs, json_file, default=datetime_converter, indent=4)

#### ad-hoc tests

In [12]:
_adc_pql = {
                    "VERSION": "v0.1.0",
                    "NAME": "Total Price of Specific Orders",
                    "DESCRIPTION": "Calculate the total price of orders with quantity greater than 10, price greater than 100, and order date between 2020-01-01 and 2021-01-01.",
                    "TABLE": "Orders_oct_12_2024_20_00",
                    "BLOCKS": [
                        {
                            "FILTER_BLOCK": {
                                "COLUMNS": [
                                    "quantity"
                                ],
                                "COMPARISON_OPERATOR": ">",
                                "VALUE": 10
                            }
                        },
                        {
                            "FILTER_BLOCK": {
                                "COLUMNS": [
                                    "price"
                                ],
                                "COMPARISON_OPERATOR": ">",
                                "VALUE": 100
                            }
                        },
                        {
                            "FILTER_BLOCK": {
                                "COLUMNS": [
                                    "date"
                                ],
                                "COMPARISON_OPERATOR": ">",
                                "VALUE": "2020-01-01"
                            }
                        },
                        {
                            "FILTER_BLOCK": {
                                "COLUMNS": [
                                    "date"
                                ],
                                "COMPARISON_OPERATOR": "<",
                                "VALUE": "2021-01-01"
                            }
                        },
                        {
                            "SCALAR_BLOCK": {
                                "COLUMNS": [
                                    "price"
                                ],
                                "OPERATOR": "COLUMN_SUM",
                                "AS": "total_price"
                            }
                        }
                    ]
                }
_adc_sql_translator = PQLTranslator(pql=_adc_pql)
_adc_sql_translator.translate()
_adc_query = _adc_sql_translator.translated_pql

filter block
{'FILTER_BLOCK': {'COLUMNS': ['quantity'], 'COMPARISON_OPERATOR': '>', 'VALUE': 10}}

            `latest_table_fc61ab067c924b1b8080a16a042d2bb8` AS (
                SELECT *
                FROM `latest_table_c5b94cb236be47f59f70f63e3ee1fc60`
                WHERE `quantity` > 10
            )
         `quantity` > 10 AND 
filter block
{'FILTER_BLOCK': {'COLUMNS': ['price'], 'COMPARISON_OPERATOR': '>', 'VALUE': 100}}

            `latest_table_b94c18acd8ac4f368891b893c2029ac5` AS (
                SELECT *
                FROM `latest_table_fc61ab067c924b1b8080a16a042d2bb8`
                WHERE `price` > 100
            )
         `price` > 100 AND 
filter block
{'FILTER_BLOCK': {'COLUMNS': ['date'], 'COMPARISON_OPERATOR': '>', 'VALUE': '2020-01-01'}}

            `latest_table_2437c5d52d394570bf37344a2da255db` AS (
                SELECT *
                FROM `latest_table_b94c18acd8ac4f368891b893c2029ac5`
                WHERE `date` > '2020-01-01'
            )
    

In [13]:
print(_adc_query)

WITH 
`latest_table_c5b94cb236be47f59f70f63e3ee1fc60` AS (
SELECT * FROM `Orders_oct_12_2024_20_00`
),

            `latest_table_fc61ab067c924b1b8080a16a042d2bb8` AS (
                SELECT *
                FROM `latest_table_c5b94cb236be47f59f70f63e3ee1fc60`
                WHERE `quantity` > 10
            )
        ,
            `latest_table_b94c18acd8ac4f368891b893c2029ac5` AS (
                SELECT *
                FROM `latest_table_fc61ab067c924b1b8080a16a042d2bb8`
                WHERE `price` > 100
            )
        ,
            `latest_table_2437c5d52d394570bf37344a2da255db` AS (
                SELECT *
                FROM `latest_table_b94c18acd8ac4f368891b893c2029ac5`
                WHERE `date` > '2020-01-01'
            )
        ,
            `latest_table_66d6c8f1219145e7b1880a253a70dfab` AS (
                SELECT *
                FROM `latest_table_2437c5d52d394570bf37344a2da255db`
                WHERE `date` < '2021-01-01'
            )
        
   